In [7]:
from tensorflow.keras import models, layers, activations, \
    optimizers, utils, losses, initializers, metrics, callbacks

In [8]:
epochs = 5
batch_size = 32
patience = 5
learning_rate = 0.001
model_path = './model/softmax.keras'

In [9]:
model = models.Sequential([
    layers.Resizing(128, 128),
    layers.Rescaling(1.0/255),
    
    layers.Conv2D(32, (5, 5),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dropout(0.2),
    layers.Dense(128,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    
    layers.Dense(86,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    
    layers.Dense(29,
        activation = 'softmax',
        kernel_initializer = initializers.RandomNormal()
    )
])

In [10]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.SparseCategoricalAccuracy() ]
)

In [11]:
train = utils.image_dataset_from_directory(
    'asl_alphabet_train',
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    'asl_alphabet_train',
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    shuffle = True,
    image_size = (512, 512),
    batch_size = batch_size
)

Found 223074 files belonging to 29 classes.
Using 178460 files for training.
Found 223074 files belonging to 29 classes.
Using 44614 files for validation.


In [12]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 1521s 272ms/step - loss: 1.3210 - sparse_categorical_accuracy: 0.6033 - val_loss: 0.1819 - val_sparse_categorical_accuracy: 0.9451
Epoch 2/100
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 726s 130ms/step - loss: 0.1517 - sparse_categorical_accuracy: 0.9535 - val_loss: 0.1005 - val_sparse_categorical_accuracy: 0.9712
Epoch 3/100
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 726s 130ms/step - loss: 0.0875 - sparse_categorical_accuracy: 0.9732 - val_loss: 0.0867 - val_sparse_categorical_accuracy: 0.9780
Epoch 4/100
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 724s 130ms/step - loss: 0.0631 - sparse_categorical_accuracy: 0.9811 - val_loss: 0.0597 - val_sparse_categorical_accuracy: 0.9852
Epoch 5/100
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 726s 130ms/step - loss: 0.0522 - sparse_categorical_accuracy: 0.9848 - val_loss: 0.0776 - val_sparse_categorical_accuracy: 0.9817
Epoch 6/100
5577/5577 ━━━━━━━━━━━━━━━━━━━━ 726s 130ms/step - loss: 0.0429 - sparse_categorical_accuracy: 0.9879 - val_loss: 0.0739 - v

In [13]:
import numpy as np
from PIL import Image
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 124, 124, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 58, 58, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 29, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 26912)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26912)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,444,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 29)             │           957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,434,041 (39.80 MB)

 Trainable params: 3,478,013 (13.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,956,028 (26.54 MB)

In [3]:
model = models.load_model('./model/softmax.keras')
model.summary()

NameError: name 'models' is not defined

In [ ]:
def load_img(img_path):
    img = Image.open(img_path)
    img_array = np.asarray(img)
    return np.expand_dims(img_array, axis=0)

In [ ]:
model.predict(load_img("d.jpeg"))